<a href="https://colab.research.google.com/github/heirynhr/legalduel-1a/blob/main/LegalDuel1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00


In [4]:

!pip install nltk


In [5]:
# importing libraries here

from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from PyPDF2 import PdfReader
from google.colab import files
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# this is how we are able to upload the different chronologies into the document
# run this cell to add new docs

uploaded = files.upload()

Saving ChronologyTest.pdf to ChronologyTest.pdf


In [10]:
# looking at our input file here, only used for looking at document, not as input
# for model

filepath = "ChronologyTest.pdf"
reader = PdfReader(filepath)

numPages = len(reader.pages)

for pageNum in range(numPages):
  page = reader.pages[pageNum]
  text = page.extract_text()
  print("Page:" + str(pageNum + 1))

print(text)

Page:1
Page:2
John
Hardwick:
They
told
me
to
see
a
doctor
for
my
injuries
and
said
they’d
investigate
the
incident.
They
also
asked
if
I
could
remember
any
details
about
the
car,
but
like
I
said,
I
didn’t
get
a
clear
look.
Attorney:
Did
you
receive
medical
treatment
following
the
incident?
John
Hardwick:
Yes,
I
went
to
the
hospital
the
next
day,
on
July
15,
2019.
They
did
X-rays
and
said
I
had
some
soft
tissue
damage
but
no
broken
bones.
Attorney:
Did
your
injuries
heal
over
time?
John
Hardwick:
Yes,
the
bruises
faded
after
a
few
weeks,
but
I
had
lingering
pain
in
my
hip
for
a
couple
of
months.
It
eventually
got
better
by
October
2019.
Attorney:
Have
you
had
any
issues
with
your
hip
since
then?
John
Hardwick:
No,
not
really.
It’s
been
fine
since
late
2019.
I
haven’t
had
any
serious
problems
with
it.
Attorney:
Thank
you,
Mr.
Hardwick.
No
further
questions
at
this
time.


In [11]:
# clean text function

def clean_text(text):
  nlpText = nlp(text)

  cleaned_text = []

  for word in nlpText:
    if not word.is_stop and not word.is_punct:
      cleaned_text.append(word.lemma_.lower())

  return " ".join(cleaned_text)

print(clean_text(text))

john 
 hardwick 
 
 tell 
 
 
 
 
 doctor 
 
 
 injury 
 
 say 
 
 investigate 
 
 incident 
 
 
 ask 
 
 
 
 remember 
 
 detail 
 
 
 car 
 
 like 
 
 say 
 
 
 
 
 clear 
 look 
 attorney 
 
 
 receive 
 medical 
 treatment 
 follow 
 
 incident 
 john 
 hardwick 
 yes 
 
 go 
 
 
 hospital 
 
 
 day 
 
 july 
 15 
 2019 
 
 
 x ray 
 
 say 
 
 
 
 soft 
 tissue 
 damage 
 
 
 break 
 bone 
 attorney 
 
 
 injury 
 heal 
 
 time 
 john 
 hardwick 
 yes 
 
 bruise 
 fade 
 
 
 
 week 
 
 
 
 linger 
 pain 
 
 
 hip 
 
 
 couple 
 
 month 
 
 eventually 
 get 
 well 
 
 october 
 2019 
 attorney 
 
 
 
 
 issue 
 
 
 hip 
 
 
 john 
 hardwick 
 
 
 
 
 
 fine 
 
 late 
 2019 
 
 
 
 
 
 problem 
 
 
 attorney 
 thank 
 
 mr. 
 hardwick 
 
 
 question 
 
 
 time


In [12]:
# extracting text for the model

def extract_text_from_pdf(filepath):
  reader = PdfReader(filepath)
  text = ""

  for i in range(len(reader.pages)):
    page = reader.pages[i]
    pageText = page.extract_text()

    pageText = pageText.replace("\n", " ").strip()
    text += pageText + "\n\n"

    final_text = clean_text(text)
  return text

In [13]:
# summarzing text function

def summarize_transcript(text, max_length=1024):
  inputs = tokenizer(text, max_length=max_length, return_tensors="pt", truncation=True)
  summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

In [15]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def create_chronology(filepath):
  transcript = extract_text_from_pdf(filepath)
  summary = summarize_transcript(transcript)
  return summary

chronology = create_chronology(filepath)
print(chronology)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Johnathan Hardwick says he was walking home from work when he was hit by a car. He says he didn't see the car coming and didn't get a good look at the vehicle. He sustained injuries to his leg and hip, but no broken bones.
